This is the Project that I've been working on while taking this [Course](https://www.udemy.com/course/tensorflow-developer-certificate-machine-learning-zero-to-mastery/learn/)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np 
import tensorflow as tf
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_dir='../input/200000-abstracts-for-seq-sentence-classification/20k_abstracts_numbers_with_@/'


In [ ]:
filenames=[data_dir + filename for filename in os.listdir(data_dir)]
filenames

<h3>Preprocessing the Data

In [ ]:
# Creating a function to read the txt Files
# This function returns all the lines in the txt file as a list
def get_lines(filename):
    with open(filename,"r") as f:
        return f.readlines()

In [ ]:
# Read the Lines in the Training Set
train_data_lines=get_lines(data_dir+"train.txt")
# train_data_lines[:20]

 <h4>
 Pre-processing the Data <br>
 List of dictionaries <br>
 Each line needs to be converted into a dictionary <br>
 Dictionary items are ordered, changeable, and does not allow duplicates.<br>
 Objectives, Methods, Results, Conclusins will all be Values to the 'TARGET' KEY<br>
 And the Text corresponding to them will be Values to the Key 'TEXT'<br>
 So each abstract would have about roughly 10-12 dicts for each statement<br>

In [ ]:
# Preprocessing Functions
# Returns a list of dictionaries of abstract's lines
# Dict Format --> {'TARGET':'Background/Results/Objetive/Concludion','Text':'The actual statement'}
def preprocess_data(filename):
    input_lines=get_lines(filename)
    #This will be used to separte the abstracts from  one another using String mets
    abstract_lines=""
    # Empty list of abstracts
    abstract_samples=[]
    for line in input_lines:
        # Check for a new abstract
        if line.startswith("###"):
            abstract_id=line
            # And since we are in a new abstract we will Reset the abstract_lines
            abstract_lines=""
        # Check for a new line \n escape seq
        elif line.isspace():
            # Split the Lines of the abstract and will return a list of one abstract
            abstract_line_split=abstract_lines.splitlines()
            # Now we have to iterate through this singular abstract
            for abstract_line_number, abstract_line in enumerate(abstract_line_split):
                #  Enumerate() method adds a counter to an iterable and returns it in a form of enumerating object.
                # Create a empty Dict per line
                line_data={}
                # Split on the tab \t esc seq
                target_text_split=abstract_line.split("\t")
                # Get the Label of the sentence as the Label
                line_data["target"]=target_text_split[0]
                # Get the Text of the Lien as the Text Key
                line_data["text"]=target_text_split[1].lower()
                # Also adding the Line Nnumber as it will also aid the model
                line_data["line_number"]=abstract_line_number
                # Number of Lines in that particular abstract
                line_data["total_lines"]=len(abstract_line_split)-1
                # Now we have to append them to the absract_samples list
                abstract_samples.append(line_data)
        # So if both the cases are not there then the line is a labelled sentence
        else:
            abstract_lines+=line
    return abstract_samples
                
            
            

In [ ]:
%%time
# Get the data and preprocess it
train_samples=preprocess_data(data_dir+"train.txt")
val_samples=preprocess_data(data_dir+"dev.txt")
test_samples=preprocess_data(data_dir+"test.txt")

In [ ]:
len(train_samples),len(val_samples),len(test_samples)

In [ ]:
# Visualizing the Data
train_samples[:20]

In [ ]:
# Now we have to turn this data into a df
train_df=pd.DataFrame(train_samples)
test_df=pd.DataFrame(test_samples)
val_df=pd.DataFrame(val_samples)

In [ ]:
train_df.head(11)

In [ ]:
# Checking the Spread of Data
train_df.target.value_counts()

In [ ]:
# Checking the Length of Lines
train_df.total_lines.plot.hist()

In [ ]:
# Get the list of just the text Columns
train_sentences=train_df["text"].tolist()
test_sentences=test_df["text"].tolist()
val_sentences=val_df["text"].tolist()

In [ ]:
len(train_sentences),len(test_sentences),len(val_sentences)

In [ ]:
# Turning the target Labels into Numeric Data
# We have 5 main labels -> Backgroun, Objective,Methods, Results, Conclusion
# We'll encode them both 1HEC and Simple Nuemrical
from sklearn.preprocessing import OneHotEncoder
# Tensorflow is incompaible with sparse matrices
one_hot_encoder=OneHotEncoder(sparse=False)
# You should reshape your X to be a 2D array not 1D array. Fitting a model requires requires a 2D array. i.e (n_samples, n_features)
train_labels_one_hot=one_hot_encoder.fit_transform(train_df["target"].to_numpy().reshape(-1,1))
val_labels_one_hot=one_hot_encoder.fit_transform(val_df["target"].to_numpy().reshape(-1,1))
test_labels_one_hot=one_hot_encoder.fit_transform(test_df["target"].to_numpy().reshape(-1,1))
train_labels_one_hot,val_labels_one_hot,test_labels_one_hot

In [ ]:
# Label Encoding also for Baseline Model
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
train_labels_encoded=le.fit_transform(train_df["target"])
test_labels_encoded=le.fit_transform(test_df["target"])
val_labels_encoded=le.fit_transform(val_df["target"])
train_labels_encoded, test_labels_encoded,val_labels_encoded

In [ ]:
# Retieving classes 
num_classes=len(le.classes_)
class_names=le.classes_
num_classes,class_names

<h3>  Model -> Naive Bayes Model </h3> <br>
<h4> TF-IDF Multinomial Naive Bayes Classifier </h4>

In [ ]:
# Turning them to tensors
# Baseline Model
# tfidf turns text into Numbers with the Formula
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
# Creating a Pipeline
# A pipeline takes Multiple Tuples
model_0=Pipeline([
    ("tf-idf",TfidfVectorizer()),
    ("clf",MultinomialNB())
    
])
model_0.fit(train_sentences,train_labels_encoded)

In [ ]:
# Evaluation and Prediction
# In scikit learn it is score for eval
model_0.score(val_sentences,val_labels_encoded)

In [ ]:
# Make Predictions for the Baseline Model
baseline_predictions=model_0.predict(val_sentences)
baseline_predictions

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
# Models for Calculating different evaluation metrics
# Returns a dict of different metrics
def calculate_results(y_true, y_pred):
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [ ]:
calculate_results(val_labels_encoded,baseline_predictions)


<h3> Conv 1D Model </h3>

In [ ]:
# Vectorize the text and then create Embeddings
from tensorflow.keras import layers
# How long is each sentence on average
sent_lens=[len(sentence.split()) for sentence in train_sentences]
avg_sent_lens=np.mean(sent_lens)
avg_sent_lens
# sent_lens
#  So we will need Padding and Truncating as the input shapes must be maintained

In [ ]:
plt.hist(sent_lens,bins=9)

In [ ]:
# Calculate the percentile of length of sentences
output_seq_len=int(np.percentile(sent_lens,95))
output_seq_len
# So 95% sentences are in length of 55

In [ ]:
# Creating a text Vectorization Layer
# Mapping our text from words to Numbers
# An embedding is a relatively low-dimensional space into which you can translate high-dimensional vectors. 
# Vocabulary size in the Research Paper is 68000
max_tokens=68000
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
text_vectorizer=TextVectorization(max_tokens=max_tokens,output_sequence_length=output_seq_len)


In [ ]:
# Adapt the Text Vectorizer to the Training Data
# We have to adapt it to only the training data so that val and test data are not seen
# Later it can be fitted to the two latter
text_vectorizer.adapt(train_sentences)

In [ ]:
# Finding out how many words are there  in the training vocabulary and which are  most common
# Also text vectorizer works pretty straightforwardly, 1 to most common word, 2 to 2nd most common word and so on
train_vocab=text_vectorizer.get_vocabulary()
# Size of Vocab
print(len(train_vocab))
# 5 Most Common Words in the Vocab
print(train_vocab[:5])
# Least common 5 words in the vocab
print(train_vocab[-5:])

In [ ]:
# Get the config of our Text Vectorizer
text_vectorizer.get_config()

In [ ]:
# Create an Embedding Layer
# More output dims , more emmbedding, more parameters to train
# Masking the 0 considering themm as padding
token_embed=layers.Embedding(input_dim=len(train_vocab),output_dim=128,mask_zero=True,name="token_embedding")

In [ ]:
# Creating a Fast Loadinng Dataset with tf data API
# https://www.tensorflow.org/guide/data_performance
# https://www.tensorflow.org/guide/data
# Turn our data into Tensorflow datasets
train_dataset=tf.data.Dataset.from_tensor_slices((train_sentences,train_labels_one_hot))
val_dataset=tf.data.Dataset.from_tensor_slices((val_sentences,val_labels_one_hot))
test_dataset=tf.data.Dataset.from_tensor_slices((test_sentences,test_labels_one_hot))
train_dataset
# <TensorSliceDataset shapes: ((), (5,)), types: (tf.string, tf.float64)>
# Which indicates one Text Sample in first tuple, next tuple is (0,0,0,0,1) -> 1hc 

In [ ]:
# Pre fetching the data and making them into batches
# Pre fetching reduces the Preparation time of Data taken by CPU
# Pref-fetching in a Multi-threaded way Reduces time and Increases the amount of data as all cores can be utilized to Prepare the Data
# The GPU will do the Computation
train_dataset=train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset=val_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset=test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
train_dataset
# Run the Previous steps as well this otherwie the Shapes will not be fixed
# PrefetchDataset shapes: ((None,), (None, 5)), types: (tf.string, tf.float64)

In [ ]:
# Creating the Model
inputs=layers.Input(shape=(1,),dtype=tf.string)
text_vectors=text_vectorizer(inputs)
token_embedding=token_embed(text_vectors)
x=layers.Conv1D(64,kernel_size=5,padding="same",activation="relu")(token_embedding)
x=layers.GlobalAveragePooling1D()(x)
outputs=layers.Dense(num_classes,activation="softmax")(x)
# Indirect way of creating the Modelling the op ip
model_1=tf.keras.Model(inputs,outputs)
# Compiling the Model
model_1.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])



In [ ]:
model_1.summary()

In [ ]:
history_model_1=model_1.fit(train_dataset,epochs=5,validation_data=val_dataset)

In [ ]:
model_1.evaluate(val_dataset)

In [ ]:
# Making Predictions In terms of Probabilities
model_1_prediction_probability=model_1.predict(val_dataset)
model_1_prediction_probability
# For all 30k statements our Model will output a 5 len list of Prediction Probability
# And out of the 5 the index that is higher is the one in which our class thinks the 
# Sentence belongs

In [ ]:
# Now converting the Probabilities to classes
model_1_prediction=tf.argmax(model_1_prediction_probability,axis=1)
model_1_prediction

In [ ]:
model_1_results=calculate_results(y_true=val_labels_encoded,y_pred=model_1_prediction)
model_1_results

<h3> Tensorflow Hub Pre trained Embedding and Feature Extractor </h3>

In [ ]:

!pip install huggingface

In [ ]:
# Model 2: Feature Extractor with pretrained token Embeddings
# This is done to leverage the power of Transfer Learning
import tensorflow_hub as hub
tf_hub_embedding_layers=hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",trainable=False,name="universal_sentence_encoder")

In [ ]:
# Creating the Model
# For the Tensorflow embedding we are using the input shape needs to be in the form of an empty list
inputs = layers.Input(shape=[], dtype=tf.string)
pretrained_embedding = tf_hub_embedding_layers(inputs)
x=layers.Dense(128,activation="relu")(pretrained_embedding)
x=layers.Dense(128,activation="relu")(x)
# Softmax because we are doing multiclass 
outputs=layers.Dense(5,activation="softmax")(x)
model_2=tf.keras.Model(inputs,outputs,name="model_2_transfer_learning")
# Compiling the Model
model_2.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])
model_2.summary()

In [ ]:
# Fitting the Model
history_model_2=model_2.fit(train_dataset,epochs=9,validation_data=val_dataset)

In [ ]:
# Evaluating on the Validation Set
model_2.evaluate(val_dataset)

In [ ]:
# Making Predictions In terms of Probabilities
model_2_prediction_probability=model_2.predict(val_dataset)
model_2_prediction_probability
# For all 30k statements our Model will output a 5 len list of Prediction Probability
# And out of the 5 the index that is higher is the one in which our class thinks the 
# Sentence belongs

In [ ]:
# Now converting the Probabilities to classes
model_2_prediction=tf.argmax(model_2_prediction_probability,axis=1)
model_2_prediction

In [ ]:
model_2_results=calculate_results(y_true=val_labels_encoded,y_pred=model_2_prediction)
model_2_results

<h3> Building a Model with Character level Embeddings </h3>

In [ ]:
# So we first need to build a Character Layer Tokenizer 
def split_chars(text):
    return " ".join(list(text))
# Text Splitting 
train_chars=[split_chars(sentence) for sentence in train_sentences]
train_chars[:5]

In [ ]:
val_chars=[split_chars(sentence) for sentence in val_sentences]
test_chars=[split_chars(sentence) for sentence in test_sentences]

In [ ]:
# Finding the Average Character Length
char_lens=[len(sentence) for sentence in train_sentences]
mean_char_len=np.mean(char_lens)
mean_char_len

In [ ]:
# The meann is not the best way to gauge so we need a distribution and should take 95% CI
plt.hist(char_lens,bins=6)

# Finding the 95% CI Length
ci_char_len=int(np.percentile(char_lens,95))
ci_char_len
# So 300 Characters must work

In [ ]:
# Setting up a Text Vectorization Layer
# Getting the vocab size
import string
alphanumeric=string.ascii_lowercase+string.digits+string.punctuation
alphanumeric

In [ ]:
# +2 for space and OOV UNK Token
max_char_vocab=len(alphanumeric)+2
char_vectorizer=TextVectorization(max_tokens=max_char_vocab,output_sequence_length=ci_char_len+10,name="char_vectorizer")
# Adapt it to the Training set of sequences
char_vectorizer.adapt(train_chars)

In [ ]:
char_vocab=char_vectorizer.get_vocabulary()
print(len(char_vocab))
char_vocab[:5],char_vocab[-5:]

In [ ]:
# Make an Embedding Layer
# As per the paper we have to make a 25 dim long Feature vector for Output dims
# Each character gets embedded into a size 25 Feature Vector
# Dont Mask Model 
char_embed=layers.Embedding(input_dim=len(char_vocab),output_dim=25,mask_zero=False,name="character_embedding")
 

In [ ]:
# Creating character level Fast Loading Datasets
train_char_dataset = tf.data.Dataset.from_tensor_slices((train_chars, train_labels_one_hot)).batch(32).prefetch(tf.data.AUTOTUNE)
val_char_dataset = tf.data.Dataset.from_tensor_slices((val_chars, val_labels_one_hot)).batch(32).prefetch(tf.data.AUTOTUNE)
val_char_dataset=tf.data.Dataset.from_tensor_slices((val_chars,val_labels_one_hot)).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
train_char_dataset

In [ ]:
# Building a BI-LSTM Model with the Character Level Embeddings
# Also experimenting with a Conv1D with Char level Embeddings
# Make Conv1D on chars only
inputs = layers.Input(shape=(1,), dtype="string")
char_vectors = char_vectorizer(inputs)
char_embeddings = char_embed(char_vectors)
x = layers.Conv1D(64, kernel_size=5, padding="same", activation="relu")(char_embeddings)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model_3 = tf.keras.Model(inputs=inputs,
                         outputs=outputs,
                         name="model_3_conv1D_char_embedding")
# Compile model
model_3.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_3.summary()

In [ ]:
model_3_history = model_3.fit(train_char_dataset,
                               epochs=6,
                               validation_data=val_char_dataset)

In the paper they have further used this BI-LSTM and then concatenated and then again passed it through another BILSTM Model.

In [ ]:
model_3.evaluate(val_char_dataset)

In [ ]:
# Making Predictions In terms of Probabilities
model_3_prediction_probability=model_3.predict(val_char_dataset)
model_3_prediction_probability
# For all 30k statements our Model will output a 5 len list of Prediction Probability
# And out of the 5 the index that is higher is the one in which our class thinks the 
# Sentence belongs

In [ ]:
# Now converting the Probabilities to classes
model_3_prediction=tf.argmax(model_3_prediction_probability,axis=1)
model_3_prediction

In [ ]:
model_3_results=calculate_results(y_true=val_labels_encoded,y_pred=model_3_prediction)
model_3_results

<h3> Building a Model with Token + Character Embedding Model </h3>
<h4> Concatenating Models 

During training, some number of layer outputs are randomly ignored or “dropped out.” This has the effect of making the layer look-like and be treated-like a layer with a different number of nodes and connectivity to the prior layer. In effect, each update to a layer during training is performed with a different “view” of the configured layer.

In [ ]:
# Building a Tf Dataset to load it faster
# Combine chars and tokens into a dataset
train_char_token_data = tf.data.Dataset.from_tensor_slices((train_sentences, train_chars)) # make data
train_char_token_labels = tf.data.Dataset.from_tensor_slices(train_labels_one_hot) # make labels
train_char_token_dataset = tf.data.Dataset.zip((train_char_token_data, train_char_token_labels)) # combine data and labels

# Prefetch and batch train data
train_char_token_dataset = train_char_token_dataset.batch(32).prefetch(tf.data.AUTOTUNE) 
# Preparing the same for Validation set
val_char_token_data = tf.data.Dataset.from_tensor_slices((val_sentences, val_chars))
val_char_token_labels = tf.data.Dataset.from_tensor_slices(val_labels_one_hot)
val_char_token_dataset = tf.data.Dataset.zip((val_char_token_data, val_char_token_labels))
val_char_token_dataset = val_char_token_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Buidling a Multi Input Model that can accpet multiple data streams
# So we will build two BI-LSTM Models one with char embeddings and token Embeddings
# Also there is going to be a dropout introduced 
# One more difference is we will be using the functional API instead or Sequential API
# What it does is that it will go through steps together and not sequentially
# So first we will crate a Token Model
token_inputs=layers.Input(shape=[],dtype=tf.string,name="token_input")
token_embeddings = tf_hub_embedding_layers(token_inputs)
token_output = layers.Dense(128, activation="relu")(token_embeddings)
token_model = tf.keras.Model(token_inputs,token_output)

In [ ]:
# Then we will create a Character Model
char_inputs = layers.Input(shape=(1,), dtype=tf.string, name="char_input")
char_vectors = char_vectorizer(char_inputs)
char_embeddings = char_embed(char_vectors)
# The one with the char embeddings is passed through a BI-LSTM Model acc. to the Paper
char_bi_lstm = layers.Bidirectional(layers.LSTM(25))(char_embeddings) 
char_model = tf.keras.Model(inputs=char_inputs, outputs=char_bi_lstm)

In [ ]:
# Concatenating both the Models as mentioned
token_char_concat = layers.Concatenate(name="token_char_hybrid")([token_model.output,char_model.output])

In [ ]:
# Creating Combined Dropout and a dense Layer and then some dropout
combined_dropout = layers.Dropout(0.5)(token_char_concat)
combined_dense = layers.Dense(200, activation="relu")(combined_dropout) 
final_dropout = layers.Dropout(0.5)(combined_dense)
# Final Output, Label Pediction Layers
output_layer = layers.Dense(num_classes, activation="softmax")(final_dropout)

In [ ]:
# 5. Construct model with char and token inputs
# 2 Inputs Token Model Input and Char Model Input 
model_4 = tf.keras.Model(inputs=[token_model.input, char_model.input],outputs=output_layer,name="model_4_token_and_char_embeddings")

In [ ]:
model_4.summary()

In [ ]:
# Plot hybrid token and character model and check if it has replicated the Paper
from tensorflow.keras.utils import plot_model
plot_model(model_4,show_shapes=True)

In [ ]:
# Compiling the Model
# None is the batch size in the shapes
model_4.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.SGD(),metrics=["accuracy"])

In [ ]:
# Fit the model on tokens and chars
model_4_history = model_4.fit(train_char_token_dataset,epochs=5,validation_data=val_char_token_dataset)

In [ ]:
# Caryying out similar exercise for Model 4
# model_4_prediction_probability=model_4.predict(val_char_dataset)
# model_4_prediction_probability
# model_4_prediction=tf.argmax(model_4_prediction_probability,axis=1)
# model_4_prediction
# model_4_results=calculate_results(y_true=val_labels_encoded,y_pred=model_4_prediction)
# model_4_results

<h3>Pretained Token Embeddings + Character Emebeddings + Positional Embeddings </h3>

In [ ]:
# Feature Engineering
# Here we will take non obvious features from the Data and encoding them Numerically to aid the model
# The order of Objectives, Methods, Results conclusion sequence is a non obvious feature
# So we need to inject them into the Model and this is a part of the Feature Engineering
# Encoding the Line Numbers
# Engineering features incorporated need to be avaialable at test time
# Positional Embeddings
train_df["line_number"].value_counts()
# Almost 10K Abstracts have 10 + Lines

In [ ]:
# Distribution
# Using TF To create one hot encoded tensors of Line Numbers
# They can be also used as it is the Lines they are but 1HC is better
train_line_numbers_one_hot=tf.one_hot(train_df["line_number"].to_numpy(),depth=15)
val_line_numbers_one_hot=tf.one_hot(val_df["line_number"].to_numpy(),depth=15)
test_line_numbers_one_hot=tf.one_hot(test_df["line_number"].to_numpy(),depth=15)
train_line_numbers_one_hot[:10],train_line_numbers_one_hot.shape

In [ ]:
# We need to this similarly for Total Lines
train_df["total_lines"].value_counts()
# We can take the Cutoff to e 20

In [ ]:
train_total_lines_one_hot=tf.one_hot(train_df["total_lines"].to_numpy(),depth=20)
val_total_lines_one_hot=tf.one_hot(val_df["total_lines"].to_numpy(),depth=20)
test_total_lines_one_hot=tf.one_hot(test_df["total_lines"].to_numpy(),depth=20)
train_total_lines_one_hot,train_total_lines_one_hot.shape

In [ ]:
# Buidling the Model
# Token Level Model + Character Level Model + Positional Models (2) -> Line Numbers and Total Lines
# Then we will Concatenate Model 1 and Model 2 
# Then Combine 3,4 and Combination of 1&2 
# Create an Output Layer
# # # Model 1 # # #
token_inputs=layers.Input(shape=[],dtype=tf.string,name="token_inputs")
token_embeddings=tf_hub_embedding_layers(token_inputs)
token_outputs=layers.Dense(128,activation="relu")(token_embeddings)
token_model=tf.keras.Model(token_inputs,token_outputs)
# Model 2 
char_inputs=layers.Input(shape=(1,),dtype=tf.string,name="char_inputs")
char_vectors=char_vectorizer(char_inputs)
char_embeddings=char_embed(char_vectors)
# 24 cos 25-26 is the vocab size and we need multiples of 8
char_bilstm=layers.Bidirectional(layers.LSTM(24))(char_embeddings)
char_model=tf.keras.Model(char_inputs,char_bilstm)

#  Model 3 and Model 4
# shape =(15,) as we have taken depth or size of line number till 15
line_number_inputs=layers.Input(shape=(15,),dtype=tf.float32,name="line_number_inputs")
x=layers.Dense(32,activation="relu")(line_number_inputs)
line_number_model=tf.keras.Model(line_number_inputs,x)

total_lines_inputs=layers.Input(shape=(20,),dtype=tf.float32,name="total_lines_inputs")
y=layers.Dense(32,activation="relu")(total_lines_inputs)
total_lines_model=tf.keras.Model(total_lines_inputs,y)

# Concatenating the 1st and 2nd Model
combined_embeddings=layers.Concatenate(name="Hybrid_Merge_12")([token_model.output,char_model.output])
# Applying the Dropouts
z=layers.Dense(256,activation="relu")(combined_embeddings)
z=layers.Dropout(0.5)(z)

# Combining all of the above 
final_embeddings=layers.Concatenate(name="char_token_positional_embeddings")([line_number_model.output,total_lines_model.output,z])

# Creating the Output Layer for accepting the above layers and giving Probabilities

output_layer=layers.Dense(5,activation="softmax",name="output_layer")(final_embeddings)
# Creating the Model
model_5=tf.keras.Model([line_number_model.input,total_lines_model.input,token_model.input,char_model.input],output_layer,name="model_5_tribrid_embedding_model")

In [ ]:
# comma pachi je hoy e batch size maate hoy shapes ma
model_5.summary()

In [ ]:
# Plot hybrid token and character model and check if it has replicated the Paper
from tensorflow.keras.utils import plot_model
plot_model(model_5,show_shapes=True)

In [ ]:
model_5.compile(loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.2),
               optimizer=tf.keras.optimizers.SGD(),
               metrics=["accuracy"])
# Label Smoothing is a regularization technique that introduces noise for the labels.

In [ ]:
# Creating Fast Loading Datasets for Model 5 with tf.data API
# ##### IMP IMP IMP IMP IMP IMP IMP IMP IMP IMP ###########
# The orders of line number, total lines, tokens, chars has been and must be maintained throughout the Model Operations
train_char_token_pos_data=tf.data.Dataset.from_tensor_slices((train_line_numbers_one_hot,train_total_lines_one_hot,train_sentences,train_chars))
train_char_token_pos_labels=tf.data.Dataset.from_tensor_slices(train_labels_one_hot)
train_char_token_pos_dataset=tf.data.Dataset.zip((train_char_token_pos_data,train_char_token_pos_labels))
train_char_token_pos_dataset=train_char_token_pos_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

val_char_token_pos_data=tf.data.Dataset.from_tensor_slices((val_line_numbers_one_hot,val_total_lines_one_hot,val_sentences,val_chars))
val_char_token_pos_labels=tf.data.Dataset.from_tensor_slices(val_labels_one_hot)
val_char_token_pos_dataset=tf.data.Dataset.zip((val_char_token_pos_data,val_char_token_pos_labels))
val_char_token_pos_dataset=val_char_token_pos_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

test_char_token_pos_data=tf.data.Dataset.from_tensor_slices((test_line_numbers_one_hot,test_total_lines_one_hot,test_sentences,test_chars))
test_char_token_pos_labels=tf.data.Dataset.from_tensor_slices(test_labels_one_hot)
test_char_token_pos_dataset=tf.data.Dataset.zip((test_char_token_pos_data,test_char_token_pos_labels))
test_char_token_pos_dataset=test_char_token_pos_dataset.batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
train_char_token_pos_dataset

In [ ]:
history_model_5 = model_5.fit(train_char_token_pos_dataset,
                              epochs=9,
                              validation_data=val_char_token_pos_dataset)


In [ ]:
# Caryying out similar exercise for Model 4
model_5_prediction_probability=model_5.predict(val_char_dataset)
model_5_prediction_probability
model_5_prediction=tf.argmax(model_5_prediction_probability,axis=1)
model_5_prediction
model_5_results=calculate_results(y_true=val_labels_encoded,y_pred=model_5_prediction)
model_5_results